### Single Agent Understanding

Note book works on the Assisstant Agent, from instantiation to experimenting

**Different tasks** may require different models, and the config_list aids in dynamically selecting the appropriate model configuration, managing API keys, endpoints, and versions for efficient operation of the intelligent assistant. In summary,

In [1]:
import autogen
from dotenv import load_dotenv

# First need to create the config_list that can be passed to the Agents. Using the dotenv route.
# Notebook oai_openai_utils.ipynb (https://github.com/Kamalabot/autogen/blob/main/notebook/oai_openai_utils.ipynb)
# provides the clear intro
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../.env',
    model_api_key_map= {
        "gpt-3.5-turbo": "OPENAI_API_KEY",
        "gpt-4": "OPENAI_API_KEY",
        "llama2":"HF_KEY",
        "codellama":"HF_KEY"
    },
    filter_dict={
        "model":{
            "gpt-3.5-turbo",
            "gpt-4",
            "llama2",
            "codellama",
        }
    }
)
config_list

[{'api_key': 'sk-888888888888888888888888888888', 'model': 'gpt-3.5-turbo'},
 {'api_key': 'sk-888888888888888888888888888888', 'model': 'gpt-4'},
 {'api_key': 'sk-00000', 'model': 'llama2'},
 {'api_key': 'sk-00000', 'model': 'codellama'}]

In [2]:
config_list01 = autogen.config_list_from_dotenv(
    dotenv_file_path="../.env",
    model_api_key_map= {
        "gpt-3.5-turbo": "OPENAI_API_KEY",
        "codellama2":{
            "api_key_env_var": "HF_KEY",
            "api_type": "local",
            "api_version": "v2",
            "base_url":"https://localhost:8888/"
        },
        "gpt-4": "OPENAI_API_KEY"
    },
    filter_dict={
        "model": {
            "gpt-3.5-turbo",
            "codellama2"
        }
    }
)
config_list01

[{'api_key': 'sk-888888888888888888888888888888', 'model': 'gpt-3.5-turbo'},
 {'api_key': 'sk-00000',
  'base_url': 'https://localhost:8888/',
  'api_type': 'local',
  'api_version': 'v2',
  'model': 'codellama2'}]

##### Starting to work with the [agentchat_oai_assistant_twoagents_basic.ipynb](https://github.com/Kamalabot/autogen/blob/main/notebook/agentchat_oai_assistant_twoagents_basic.ipynb) notebook

In [2]:
import logging 
import os
from autogen import AssistantAgent
from autogen import UserProxyAgent
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent

In [4]:
gpt_35_config = autogen.config_list_from_dotenv(
    dotenv_file_path="../.env",
    model_api_key_map={
        "gpt-3.5-turbo-1106":"OPENAI_API_KEY",
    },
    filter_dict={
        "model":{
            "gpt-3.5-turbo-1106"       
        }
    }
)
gpt_35_config

[{'api_key': 'sk-888888888888888888888888888888',
  'model': 'gpt-3.5-turbo-1106'}]

#### GPTAssistant initiation variables

name (str): name of the agent.
            instructions (str): instructions for the OpenAI assistant configuration.
            When instructions is not None, the system message of the agent will be
            set to the provided instructions and used in the assistant run, irrespective
            of the overwrite_instructions flag. But when instructions is None,
            and the assistant does not exist, the system message will be set to
            AssistantAgent.DEFAULT_SYSTEM_MESSAGE. If the assistant exists, the
            system message will be set to the existing assistant instructions.

llm_config (dict or False): llm inference configuration.

                - assistant_id: ID of the assistant to use. If None, a new assistant will be created.

                - model: Model to use for the assistant (gpt-4-1106-preview, gpt-3.5-turbo-1106).

                - check_every_ms: check thread run status interval

                - tools: Give Assistants access to OpenAI-hosted tools like Code Interpreter and Knowledge Retrieval,
                        or build your own tools using Function calling. ref https://platform.openai.com/docs/assistants/tools

                - file_ids: files used by retrieval in run

overwrite_instructions (bool): whether to overwrite the instructions of an existing assistant.

In [21]:
print(AssistantAgent.DEFAULT_SYSTEM_MESSAGE)

You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feed

In [22]:
print(AssistantAgent.DEFAULT_CONFIG)

{'model': 'gpt-4'}


#### [Learning about the assistant](https://platform.openai.com/docs/assistants/overview) from Openai

* Helps to build AI assistants within your own applications.

* Leverages models, tools and knowledge to respond to user queries.

* Three types of tools: Code Interpreter, Retrieval, and Function calling. 

* Assistants can access multiple tools in parallel, access persistent threads and access files in several formats.

#### Steps involved in using the Assistant

1 - Creating the Assistant using the above steps

2 - Create a thread inside OpenAI environment using thread = client.beta.threads.create()

Threads and Messages represent a conversation session between an Assistant and a user. There is no limit to the number of Messages you can store in a Thread

3 - Add message to the thread

4 - Assistants can be made to respond by initiating a Run using 

run = client.beta.threads.runs.create(
 
  thread_id=thread.id,
 
  assistant_id=assistant.id,
 
  instructions="Please address the user as Jane Doe. The user has a premium account."

)

5 - Check the run status using  client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

6 - messages = client.beta.threads.messages.list(thread_id=thread.id)

#### Upload a file with an "assistants" purpose

assistant = client.beta.assistants.create(

  instructions="You are a personal math tutor. When asked a math question, write and run code to answer the question.",

  model="gpt-4-1106-preview",

  tools=[{"type": "code_interpreter"}]

)

file = client.files.create(

  file=open("speech.py", "rb"),

  purpose='assistants'

)

#### Create an assistant using the file ID

assistant = client.beta.assistants.create(

  instructions="You are a personal math tutor. When asked a math question, write and run code to answer the question.",

  model="gpt-4-1106-preview",

  tools=[{"type": "code_interpreter"}],

  file_ids=[file.id]

)
#### Creating Threads

thread = client.beta.threads.create(
 
  messages=[
 
    {
 
      "role": "user",
 
      "content": "I need to solve the equation `3x + 11 = 14`. Can you help me?",
 
      "file_ids": [file.id]
 
    }])

#### Knowledge Retrieval 

Upload a file with an "assistants" purpose

file = client.files.create(
 
  file=open("knowledge.pdf", "rb"),
 
  purpose='assistants'

)

assistant = client.beta.assistants.create(
 
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
 
  model="gpt-4-1106-preview",
 
  tools=[{"type": "retrieval"}]

)

#### Function calling

assistant = client.beta.assistants.create(
  
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  
  model="gpt-4-1106-preview",
  
  tools=[{
  
      "type": "function",
  
    "function": {
  
      "name": "getCurrentWeather",
  
      "description": "Get the weather in location",
  
      "parameters": {
  
        "type": "object",
  
        "properties": {
  
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
  
          "unit": {"type": "string", "enum": ["c", "f"]}
  
        },
  
        "required": ["location"]}}}

In [ ]:
load_dotenv()
asst_id = os.environ.get('ASST_ID')
asst_id

In [5]:
# Ensure that Openai library is updated before working on the assistants
# If the assistant id is not set, then each run of the cell creates a new assistant inside OpenAI
# The above assistant is created inside the OpenAI backend, which can be viewed at [assistant page](https://platform.openai.com/assistants)
llm_config = {
    "config_list": gpt_35_config,
    "assistant_id": asst_id,
    "model":"gpt-3.5-turbo",
}

gpt_assistant = GPTAssistantAgent(name="assistant",
                                  instructions=AssistantAgent.DEFAULT_SYSTEM_MESSAGE,
                                  llm_config=llm_config)


#### Understand UserProxy Agent Init parameters

name (str): name of the agent.

is_termination_msg (function): a function that takes a message in the form of a dictionary
    and returns a boolean value indicating if this received message is a termination message.
    The dict can contain the following keys: "content", "role", "name", "function_call".

max_consecutive_auto_reply (int): the maximum number of consecutive auto replies.
    default to None (no limit provided, class attribute MAX_CONSECUTIVE_AUTO_REPLY will be used as the limit in this case).
    The limit only plays a role when human_input_mode is not "ALWAYS".

human_input_mode (str): whether to ask for human inputs every time a message is received.
    Possible values are "ALWAYS", "TERMINATE", "NEVER".

    (1) When "ALWAYS", the agent prompts for human input every time a message is received.
        Under this mode, the conversation stops when the human input is "exit",
        or when is_termination_msg is True and there is no human input.

    (2) When "TERMINATE", the agent only prompts for human input only when a termination message is received or
        the number of auto reply reaches the max_consecutive_auto_reply.

    (3) When "NEVER", the agent will never prompt for human input. Under this mode, the conversation stops
        when the number of auto reply reaches the max_consecutive_auto_reply or when is_termination_msg is True.

function_map (dict[str, callable]): Mapping function names (passed to openai) to callable functions.

code_execution_config (dict or False): config for the code execution.
    To disable code execution, set to False. Otherwise, set to a dictionary with the following keys:

    - work_dir (Optional, str): The working directory for the code execution.
        If None, a default working directory will be used.
        The default working directory is the "extensions" directory under
        "path_to_autogen".

    - use_docker (Optional, list, str or bool): The docker image to use for code execution.
        If a list or a str of image name(s) is provided, the code will be executed in a docker container
        with the first image successfully pulled.
        If None, False or empty, the code will be executed in the current environment.
        Default is True, which will be converted into a list.
        If the code is executed in the current environment,
        the code must be trusted.

    - timeout (Optional, int): The maximum execution time in seconds.

    - last_n_messages (Experimental, Optional, int): The number of messages to look back for code execution. Default to 1.

default_auto_reply (str or dict or None): the default auto reply message when no code execution or llm based reply is generated.

llm_config (dict or False): llm inference configuration.
    Please refer to [OpenAIWrapper.create](/docs/reference/oai/client#create)
    for available options.
    Default to false, which disables llm-based auto reply.

system_message (str): system message for ChatCompletion inference.
    Only used when llm_config is not False. Use it to reprogram the agent.Args:

In [6]:
# GPTAssistant has been understood, now move to UserProxyAgent

user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config={
        "work_dir":"auto_code"
    },
    is_termination_msg= lambda msg: "TERMINATE" in msg["content"],
    human_input_mode='NEVER',
    max_consecutive_auto_reply=1
)

In [7]:
user_proxy.initiate_chat(gpt_assistant, message="Say hello to the world")

user_proxy (to assistant):

Say hello to the world

--------------------------------------------------------------------------------
assistant (to user_proxy):

To say hello to the world, you can simply print the string "Hello, world!" in Python. Here's the code:

```python
print("Hello, world!")
```

Run this code and it will output "Hello, world!" to the console.


```python
print("Hello, world!")
```

This code will print "Hello, world!" to the console.


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...



>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is python)...


user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Hello, world!

Hello, world!


--------------------------------------------------------------------------------
assistant (to user_proxy):

Great! The code executed successfully and printed "Hello, world!" twice. The output suggests that the code was run twice. Please make sure to run the code only once to avoid duplicate outputs. If the issue persists, let me know and I'll be happy to assist you further.


TERMINATE


--------------------------------------------------------------------------------


In [8]:
user_proxy.initiate_chat(gpt_assistant, message="Write a python code to eval 2 * 20", clear_history=False)

user_proxy (to assistant):

Write a python code to eval 2 * 20

--------------------------------------------------------------------------------
assistant (to user_proxy):

Apologies for the error. Please use the following Python code to evaluate the expression "2 * 20":

```python
result = 2 * 20
print(result)
```

This code will compute the expression `2 * 20` and print the result, which should be `40`.


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
40


--------------------------------------------------------------------------------
assistant (to user_proxy):

Great! The code executed successfully and the output is `40`, which is the result of evaluating `2 * 20`. This means that the expression `2 * 20` is equal to 40. If you have any more questions, feel free to ask.


--------------------------------------------------------------------------------


In [9]:
# Lets check what has happenend inside OpenAI

gpt_assistant.assistant_id

'asst_iHWfEf9BtNKEzC6Z0jH8AVwo'

In [10]:
gpt_assistant.client

In [30]:
gpt_assistant._openai_threads

{<autogen.agentchat.user_proxy_agent.UserProxyAgent at 0x295ae42da90>: Thread(id='thread_D0EaOAFCELSV626U6H11guG5', created_at=1700132022, metadata={}, object='thread')}

In [31]:
gpt_assistant._oai_messages

defaultdict(list,
            {<autogen.agentchat.user_proxy_agent.UserProxyAgent at 0x295ae42da90>: [{'content': 'Say hello to the world',
               'role': 'user'},
              {'content': 'To say hello to the world, you can simply print the string "Hello, world!" in Python. Here\'s the code:\n\n```python\nprint("Hello, world!")\n```\n\nRun this code and it will output "Hello, world!" to the console.\n\n\n```python\nprint("Hello, world!")\n```\n\nThis code will print "Hello, world!" to the console.\n',
               'role': 'assistant'},
              {'content': 'exitcode: 0 (execution succeeded)\nCode output: \nHello, world!\n\nHello, world!\n',
               'role': 'user'},
              {'content': 'Great! The code executed successfully and printed "Hello, world!" twice. The output suggests that the code was run twice. Please make sure to run the code only once to avoid duplicate outputs. If the issue persists, let me know and I\'ll be happy to assist you further.\n\n\n

In [11]:
from openai import OpenAI
open_ai_key = os.environ.get('OPENAI_API_KEY')
client = OpenAI(api_key=open_ai_key)

In [16]:
# get the assistants
client.beta.assistants.list()

SyncCursorPage[Assistant](data=[Assistant(id='asst_iHWfEf9BtNKEzC6Z0jH8AVwo', created_at=1700130823, description=None, file_ids=[], instructions='You are a helpful AI assistant.\nSolve tasks using your coding and language skills.\nIn the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\n    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.\n    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.\nSolve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code

In [19]:
assitantused = client.beta.assistants.retrieve(assistant_id=asst_id)

In [21]:
assitantused.name

'assistant'

In [23]:
print(assitantused.instructions)

You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feed

In [ ]:
client.beta.threads.retrieve(thread_id="thread_D0EaOAFCELSV626U6H11guG5")

In [34]:
runs = client.beta.threads.runs.list(thread_id="thread_D0EaOAFCELSV626U6H11guG5")

In [ ]:
runs_data = runs.data
runs_data